In [ ]:
!pip install -U transformers datasets accelerate peft bitsandbytes
!pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#Load a Model with Quantization (4-bit or 8-bit)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#Step 3: Apply PEFT (LoRA)
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [ ]:
#Step 4: Load a Hugging Face Dataset
from datasets import load_dataset

dataset = load_dataset("yelp_review_full", split="train[:1%]")
dataset = dataset.map(lambda x: {"text": f"Review: {x['text']}\nSentiment: {x['label']}"})

In [ ]:
# Original format before your mapping:-
# {
#   "text": "the food was great and service was fast",
#   "label": 4  # integer class label (0 to 4 for stars 1–5)
# }

In [ ]:
# After Mapping:
# lambda x: {"text": f"Review: {x['text']}\nSentiment: {x['label']}"}

In [ ]:
# {
#   "text": "Review: the food was great and service was fast\nSentiment: 4"
# }

In [ ]:
# Step 5: Tokenize Dataset
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds.set_format("torch")

In [ ]:
#Step 6: Fine-tune the Model
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_dir="./logs",
    fp16=True,
    save_strategy="epoch",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
500,2.756000
1000,2.689700
1500,2.681800


TrainOutput(global_step=1625, training_loss=2.7077075383112983, metrics={'train_runtime': 512.6694, 'train_samples_per_second': 12.679, 'train_steps_per_second': 3.17, 'total_flos': 6041196036096000.0, 'train_loss': 2.7077075383112983, 'epoch': 1.0})

In [ ]:
# Step 7: Test the Fine-Tuned Model
model.eval()

input_text = "Review: The food was Good\nSentiment:"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Review: The food was Good
Sentiment: 3.5 based on 1 reviews
Sentiment


In [ ]:
#Save Fine-Tuned Model
model.save_pretrained("my-finetuned-model")
tokenizer.save_pretrained("my-finetuned-model")

('my-finetuned-model/tokenizer_config.json',
 'my-finetuned-model/special_tokens_map.json',
 'my-finetuned-model/vocab.json',
 'my-finetuned-model/merges.txt',
 'my-finetuned-model/added_tokens.json',
 'my-finetuned-model/tokenizer.json')

In [ ]:
# Step 1: Install and Login to Hugging Face CLI
!pip install -U huggingface_hub

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
hf_repo_name = "fine-tuned-falcon-rw1b-peft_abu"  # Choose your own name

In [ ]:
#Step 3: Push the Fine-Tuned Model
from huggingface_hub import HfApi
from peft import PeftModel

# Make sure your tokenizer and model are saved locally
model.save_pretrained(hf_repo_name)
tokenizer.save_pretrained(hf_repo_name)

# (Optional if using PEFT) Save adapter separately
# If you used `get_peft_model`, this will save just the LoRA weights
if isinstance(model, PeftModel):
    model.push_to_hub(hf_repo_name)
else:
    from transformers import AutoModel
    AutoModel.save_pretrained(model, hf_repo_name)

# Push tokenizer
tokenizer.push_to_hub(hf_repo_name)


adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ucool38/fine-tuned-falcon-rw1b-peft_abu/commit/b691120eeef2bba70af2abca5f9661d33adeeb63', commit_message='Upload tokenizer', commit_description='', oid='b691120eeef2bba70af2abca5f9661d33adeeb63', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ucool38/fine-tuned-falcon-rw1b-peft_abu', endpoint='https://huggingface.co', repo_type='model', repo_id='ucool38/fine-tuned-falcon-rw1b-peft_abu'), pr_revision=None, pr_num=None)